# Collect RAWS Data with SynopticPy

## Setup

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

from synoptic.services import stations_timeseries, stations_metadata

C:\Users\jhirs\anaconda3\lib\site-packages\synoptic\accessors.py:28: UserWarning: map making not available without Brian's cartopy_tools
  warnings.warn("map making not available without Brian's cartopy_tools")
C:\Users\jhirs\anaconda3\lib\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn(


## Station Time Series

In [25]:
params = dict(
    stid=["KAFF"],
    vars=["air_temp", "relative_humidity", "precip_accum", "fuel_moisture", "wind_speed", "solar_radiation"],
    start=datetime(2023, 2, 1),
    end=datetime(2024, 1, 31),
)

a = stations_timeseries(**params)

a


 🚚💨 Speedy Delivery from Synoptic API [timeseries]: https://api.synopticdata.com/v2/stations/timeseries?stid=KAFF&vars=air_temp,relative_humidity,precip_accum,fuel_moisture,wind_speed,solar_radiation&start=202302010000&end=202401310000&token=🙈HIDDEN



,air_temp,relative_humidity,wind_speed
date_time,,,
2023-02-26 20:55:00+00:00,10.7,19.91,11.318
2023-02-26 21:24:00+00:00,12.0,17.41,5.659
2023-02-26 21:55:00+00:00,11.2,21.88,4.630
2023-02-26 22:09:00+00:00,11.0,21.82,6.688
2023-02-26 22:55:00+00:00,10.2,19.94,9.260
...,...,...,...
2024-01-30 19:55:00+00:00,14.2,22.03,2.058
2024-01-30 20:55:00+00:00,15.2,16.99,0.000
2024-01-30 21:55:00+00:00,15.1,24.04,4.630


In [18]:
type(a)

pandas.core.frame.DataFrame

In [4]:
print(f'Station Elevation: {a.attrs["ELEVATION"]}')
print(f'Lat: {a.attrs["latitude"]}')
print(f'Lon: {a.attrs["longitude"]}')
print(f'Temp Units: {a.attrs["UNITS"]["air_temp"]}')

Station Elevation: 8280
Lat: 40.6125
Lon: -111.09833
Temp Units: Celsius


In [5]:
a.attrs

{'ID': 1496,
 'STID': 'NWRU1',
 'NAME': 'NORWAY',
 'ELEVATION': 8280,
 'STATUS': 'ACTIVE',
 'MNET_ID': 2,
 'STATE': 'UT',
 'TIMEZONE': 'America/Denver',
 'ELEV_DEM': 8251.3,
 'PERIOD_OF_RECORD': {'start': '1998-07-28T00:00:00Z',
  'end': '2024-02-26T17:04:00Z'},
 'UNITS': {'position': 'm',
  'elevation': 'ft',
  'precip_accum': 'Millimeters',
  'solar_radiation': 'W/m**2',
  'air_temp': 'Celsius',
  'fuel_moisture': 'gm',
  'relative_humidity': '%',
  'wind_speed': 'm/s'},
 'QC_FLAGGED': False,
 'RESTRICTED': False,
 'RENAMED': {'air_temp_set_1': 'air_temp',
  'fuel_moisture_set_1': 'fuel_moisture',
  'precip_accum_set_1': 'precip_accum',
  'relative_humidity_set_1': 'relative_humidity',
  'solar_radiation_set_1': 'solar_radiation',
  'wind_speed_set_1': 'wind_speed'},
 'latitude': 40.6125,
 'longitude': -111.09833,
 'SUMMARY': {'NUMBER_OF_OBJECTS': 1,
  'RESPONSE_CODE': 1,
  'RESPONSE_MESSAGE': 'OK',
  'METADATA_RESPONSE_TIME': '234.8 ms',
  'DATA_QUERY_TIME': '6.2 ms',
  'QC_QUERY_TI

In [6]:
a.attrs["UNITS"]["air_temp"] == "Celsius"

True

In [7]:
a.attrs["UNITS"]['elevation']

'ft'

## Get Data For Bounding Box

For any moderate large bounding box, the package returns errors that too many station hours were requested. We will explore getting a paid token to get around this, but for now we will loop over available stations in the bounding box to collect data.

In [ ]:
box = [-105, 37, -103, 39]
# Find all stations with fuel moisture
sts = stations_metadata(
    bbox=box,
    vars=["fuel_moisture"]
)

In [27]:
sts

,KAFF,KCOS,KFCS,KLHX,KPUB,KTAD,FCRC2,CCEC2,TMPE5,TR048,...,F3769,C1234,CSA01,CSA02,A4902,G3678,G3727,G4226,FNEC2,JCFC2
ELEVATION,6572.0,6186.0,5842.0,4193.0,4672.0,5738.0,6450.0,5422.0,5791.0,6050.0,...,1561.0,6845.0,5448.0,5341.0,5128.0,5088.0,6646.0,6108.0,5921.0,5554.0
ELEV_DEM,6525.6,6177.8,5846.5,4202.8,4678.5,5738.2,6463.3,5393.7,5777.6,6063.0,...,NaN,NaN,NaN,NaN,NaN,NaN,6653.5,NaN,NaN,NaN
ID,86,140,186,243,337,410,1974,1990,1993,2891,...,194451,196181,199828,199829,217330,218898,219503,231004,231413,231485
MNET_ID,1,1,1,1,1,1,2,2,1008,2,...,65,65,221,221,136,65,65,65,106,106
NAME,Air Force Academy,City of Colorado Springs Municipal Airport,Butts Army Airfield (Fort Carson),La Junta Municipal Airport,Pueblo Memorial Airport,Perry Stokes Airport,FT CARSON,PINION CANYON,RED CREEK,ARINC TEST,...,FW3769 Gartringen,CW1234 Breitenstein,Pueblo West,Punkin Center,Pueblo West,GW3678 PUEBLO WEST,GW3727 COLO SPRINGS,GW4226 COLORADO SPRINGS,FOUNTAIN CREEK AT COLORADO SPRINGS,JIMMY CAMP CREEK NEAR FOUNTAIN 1E
RECORD_END,2024-02-27 00:55:00+00:00,2024-02-27 00:55:00+00:00,2024-02-27 00:55:00+00:00,2024-02-27 00:55:00+00:00,2024-02-27 00:55:00+00:00,2024-02-27 00:55:00+00:00,2024-02-27 00:58:00+00:00,2024-02-27 00:23:00+00:00,2001-04-03 22:15:00+00:00,2011-11-17 15:49:00+00:00,...,2024-02-27 01:00:00+00:00,2024-02-27 00:59:00+00:00,2024-02-27 01:00:00+00:00,2024-02-27 01:00:00+00:00,2024-02-27 00:00:00+00:00,2024-02-27 01:00:00+00:00,2024-02-27 01:00:00+00:00,2024-02-27 00:55:00+00:00,2024-01-14 20:45:00+00:00,2024-02-27 00:45:00+00:00
RECORD_START,1997-01-02 00:00:00+00:00,1997-01-01 00:00:00+00:00,1997-09-15 00:00:00+00:00,1997-04-11 00:00:00+00:00,1997-01-01 00:00:00+00:00,1997-01-17 00:00:00+00:00,1998-07-29 00:00:00+00:00,1998-07-29 00:00:00+00:00,2000-03-24 00:00:00+00:00,2001-04-03 00:00:00+00:00,...,2023-01-25 16:06:00+00:00,2023-01-30 16:05:00+00:00,2023-02-15 07:03:00+00:00,2023-02-14 22:03:00+00:00,2023-05-11 20:25:00+00:00,2023-07-05 14:20:00+00:00,2023-08-01 17:03:00+00:00,2023-12-12 19:55:00+00:00,2024-01-12 22:45:00+00:00,2024-01-23 14:45:00+00:00
RESTRICTED,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
STATE,CO,CO,CO,CO,CO,CO,CO,CO,CO,CO,...,CO,CO,CO,CO,CO,CO,CO,CO,CO,CO
STATUS,ACTIVE,ACTIVE,ACTIVE,ACTIVE,ACTIVE,ACTIVE,ACTIVE,ACTIVE,INACTIVE,INACTIVE,...,ACTIVE,ACTIVE,ACTIVE,ACTIVE,ACTIVE,ACTIVE,ACTIVE,ACTIVE,INACTIVE,ACTIVE


In [28]:
params = dict(
    stid=["PLACEHOLDER"], # change this in the loop
    vars=["air_temp", "relative_humidity", "precip_accum", "fuel_moisture", "wind_speed", "solar_radiation"],
    start=datetime(2023, 6, 1),
    end=datetime(2023, 6, 30),
)

In [39]:
# Set up final data dictionary
raws_dict = {key: {} for key in list(sts.columns)}

In [ ]:
for st in sts:
    print("~"*50)
    print(f"Collecting data for {st}")
    params["stid"] = [st]
    try:
        dat = stations_timeseries(verbose="HIDE", **params)
    except AssertionError as e:
        # Error handling behavior
        print("AssertionError caught:", e)
        # Other behavior you want to execute if AssertionError is caught

    print(dat.columns)

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

 🚚💨 Speedy Delivery from Synoptic API [timeseries]: https://api.synopticdata.com/v2/stations/timeseries?stid=KAFF&vars=air_temp,relative_humidity,precip_accum,fuel_moisture,wind_speed,solar_radiation&start=202306010000&end=202306300000&token=🙈HIDDEN

Index(['air_temp', 'relative_humidity', 'wind_speed'], dtype='object')
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

 🚚💨 Speedy Delivery from Synoptic API [timeseries]: https://api.synopticdata.com/v2/stations/timeseries?stid=KCOS&vars=air_temp,relative_humidity,precip_accum,fuel_moisture,wind_speed,solar_radiation&start=202306010000&end=202306300000&token=🙈HIDDEN

Index(['air_temp', 'relative_humidity', 'wind_speed'], dtype='object')
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

 🚚💨 Speedy Delivery from Synoptic API [timeseries]: https://api.synopticdata.com/v2/stations/timeseries?stid=KFCS&vars=air_temp,relative_humidity,precip_accum,fuel_moisture,wind_speed,solar_radiation&